In [1]:

from    acevedo_clss_and_fcns import * 
#import  modin.pandas as pd
from    cobra.io.mat import *



model = load_matlab_model("./COBRA_models/GEM_Recon3_thermocurated_redHUMAN_AA.mat")
feature_data                          = pd.read_parquet("./results/dataframes/augmented_metabolite_data_v2.parquet.gzip")#
feature_names                         = pd.read_csv("./metabolites_data/metabolite_names.csv")
grafo_nx             = cobra_to_networkx(model)




/DeepenData/.miniconda/envs/geo/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


In [2]:
from networkx.algorithms import bipartite
first_partition , second_partition = bipartite.sets(grafo_nx)
second_partition

{'r0557',
 'HMR_3054',
 'CHSTEROLt1',
 'EX_doco13ac_e',
 'ARGARGMETt',
 'N3Tg',
 'UGT1A2r',
 'GBGT1',
 'r1525',
 'EX_so4_e',
 'r1810',
 'STCRNtr',
 'HMR_0259',
 'PRGNLONESULT',
 'LCA3St',
 'EX_CE2915_e',
 'HMR_3448',
 'SMVACIDATPteb',
 'CLPNDCOAtx',
 'HMR_3352',
 'XOL27OHtmc',
 'C6DCc',
 'BALABETAtc',
 'NACASPAH',
 'ASPPROASCT1',
 'OCTDECE1CRNe',
 'HMR_2343',
 'r1947',
 'ASPPROLYSr',
 'r0637',
 'r0365',
 'r2230',
 'HMR_0241',
 'DM_pe_hs_c',
 'O16G2e',
 'DNDPt20m',
 '34DHOXPEGOX',
 'HMR_4831',
 '3MLDAt',
 '15DMTtu',
 'MTHFR3',
 'DM_na1_x',
 '7THFtl',
 'AM1ACShr',
 'OMEPRAZOLEte',
 'HCA24GSc',
 'ACMPGLUtep',
 'EX_core4_e',
 'SPHMYLN181181_hs_ATPt',
 'r0954',
 'EX_CE2445_e',
 'EX_hgentis_e',
 'RE3016R',
 'PCHOLLINL_HSABCt',
 '13HTACRitr',
 'HCA6Gte',
 'r2071',
 'DNDPt12m',
 'HMR_3172',
 'TRPTYRTYRr',
 '34DHPLACOX_NADP',
 'THYMDtm',
 'RE2030M',
 'LEUVALr',
 'DNDPt63m',
 'EX_sphmyln181181_hs_e',
 'FUC13GALACGLCGAL14ACGLCGALGLUSIDEtg',
 'HMR_2945',
 'HMR_0209',
 'HMR_3178',
 'PCHOLSTE_HSt1e'

In [3]:


feature_data['Leu'] = feature_data['Leu.Ile']
feature_data['Ile'] = feature_data['Leu.Ile']
feature_data.drop('Leu.Ile', axis=1, inplace=True)

feature_data['C3DC'] = feature_data['C4OH.C3DC']
feature_data['C4OH'] = feature_data['C4OH.C3DC']
feature_data.drop('C4OH.C3DC', axis=1, inplace=True)

feature_data['C4DC'] = feature_data['C5.OH.C4DC']
feature_data['C5OH'] = feature_data['C5.OH.C4DC']
feature_data.drop('C5.OH.C4DC', axis=1, inplace=True)
assert len(set(feature_names.Simbolo_traductor) - set(feature_data.columns)) == 0




node_list = list(grafo_nx.nodes)
feature_data.rename(
        columns=feature_names.set_index("Simbolo_traductor")["Recon3_ID"].to_dict(), 
        inplace=True
    )

assert set(set(feature_data.columns)-set(["label"])).issubset(set(list(grafo_nx.nodes)))


In [4]:
w  = dict(zip(grafo_nx.edges() , itertools.repeat(1)))

nx.set_edge_attributes(grafo_nx, w, "weight")

assert 1 == np.unique(list(nx.get_edge_attributes(grafo_nx, "weight").values())).__len__()

In [5]:
#grafo_nx.nodes.__len__()

flux_samples_CONTROL_7_000  = pd.read_parquet("./results/fluxes/flux_samples_CONTROL_7_000.parquet.gzip")
flux_samples_PKU_7_000      = pd.read_parquet("./results/fluxes/flux_samples_PKU_7_000.parquet.gzip")




In [6]:
def get_sample_subset(full_samples, feature_data, label): 


    s = feature_data.label.value_counts()

    sample_subset = full_samples.sample(s.loc[label], replace=True).reset_index(drop=True)

    sample_subset["label"] = label
    
    return sample_subset
    
    
flux_samples_CONTROL = get_sample_subset(flux_samples_CONTROL_7_000, feature_data, 0)
flux_samples_PKU     = get_sample_subset(flux_samples_PKU_7_000,     feature_data, 1)



assert flux_samples_CONTROL.r0399.max() > 30 
assert flux_samples_PKU.r0399.max() < 1



#pd.concat([flux_samples_CONTROL, flux_samples_PKU], axis=1)

all(flux_samples_CONTROL.columns == flux_samples_PKU.columns)



flux_samples_CONTROL = flux_samples_CONTROL.reindex(columns=flux_samples_PKU.columns)



flux_samples = pd.concat([flux_samples_CONTROL, flux_samples_PKU], axis=0)
flux_samples = flux_samples.reset_index(drop=True, inplace=False)

assert len(flux_samples.columns) == len(flux_samples_CONTROL.columns)
assert len(feature_data) == len(flux_samples)
assert flux_samples.r0399.loc[flux_samples.label == 0].mean() > 30
assert flux_samples.r0399.loc[flux_samples.label == 1].mean() < 1



In [7]:


def get_largest_cc(G):
    
  largest_wcc = max(nx.connected_components(nx.Graph(G)), key=len)


  # Create a subgraph SG based on G
  SG = G.__class__()
  SG.add_nodes_from((n, G.nodes[n]) for n in largest_wcc)


  SG.add_edges_from((n, nbr, d)
      for n, nbrs in G.adj.items() if n in largest_wcc
      for nbr, d in nbrs.items() if nbr in largest_wcc)

  SG.graph.update(G.graph)

  assert G.nodes.__len__() >= SG.nodes.__len__()
  assert G.edges.__len__() >= SG.edges.__len__()
  assert SG.nodes.__len__() == len(largest_wcc)
  #assert not SG.is_directed() 
  assert nx.is_connected(nx.Graph(SG))

  return copy.deepcopy(SG)
#to_remove = pd.read_csv('./results/dataframes/to_remove.csv').loc[:,"0"].tolist()



mets = ['nad_', 'nadh_', "nadp_", "nadph_", "adp_", "atp_", "gdt_", "gtp_",
        "pi_", "ppi_", "pppi_", "co2_", "hco3_", "h2o_", "h2o2_", "h_", "o2_", "oh1_",
        "o2s_", "fad_",  "fadh2_", "nh4_", "so3_", "so4_", "cl_", "k_", "na1_",
        "i_", "fe2_", "fe3_", "mg2_", "ca2", "zn2_", "M02382_"]

to_remove = []
for m in mets:

    to_remove.extend(["".join(l) for l in list(zip(itertools.repeat(m), list(model.compartments.keys())))])


In [8]:

grafo_nx.remove_nodes_from(to_remove)

grafo_nx = get_largest_cc(grafo_nx)
first_partition , second_partition = bipartite.sets(grafo_nx)



{'r0557',
 'HMR_3054',
 'CHSTEROLt1',
 'EX_doco13ac_e',
 'ARGARGMETt',
 'UGT1A2r',
 'N3Tg',
 'GBGT1',
 'r1525',
 'r1810',
 'STCRNtr',
 'HMR_0259',
 'PRGNLONESULT',
 'LCA3St',
 'EX_CE2915_e',
 'HMR_3448',
 'SMVACIDATPteb',
 'CLPNDCOAtx',
 'C6DCc',
 'HMR_3352',
 'XOL27OHtmc',
 'BALABETAtc',
 'NACASPAH',
 'ASPPROASCT1',
 'OCTDECE1CRNe',
 'HMR_2343',
 'r1947',
 'ASPPROLYSr',
 'r0637',
 'r0365',
 'r2230',
 'HMR_0241',
 'DM_pe_hs_c',
 'DNDPt20m',
 'O16G2e',
 '34DHOXPEGOX',
 'HMR_4831',
 '3MLDAt',
 '15DMTtu',
 'MTHFR3',
 '7THFtl',
 'AM1ACShr',
 'HCA24GSc',
 'ACMPGLUtep',
 'EX_core4_e',
 'SPHMYLN181181_hs_ATPt',
 'r0954',
 'EX_CE2445_e',
 'EX_hgentis_e',
 'RE3016R',
 'PCHOLLINL_HSABCt',
 '13HTACRitr',
 'HCA6Gte',
 'r2071',
 'DNDPt12m',
 'HMR_3172',
 'TRPTYRTYRr',
 '34DHPLACOX_NADP',
 'THYMDtm',
 'RE2030M',
 'LEUVALr',
 'DNDPt63m',
 'EX_sphmyln181181_hs_e',
 'FUC13GALACGLCGAL14ACGLCGALGLUSIDEtg',
 'HMR_2945',
 'HMR_0209',
 'HMR_3178',
 'PCHOLSTE_HSt1e',
 'GLYGLYCNc',
 'r2410',
 'S6TASE7ly',
 'U

In [10]:

first_partition

{'achom_L_m',
 'CE0693_x',
 'stcoa_c',
 '3hdececrn_e',
 '11docrtsl_r',
 'crvnc_c',
 'acnacngalgbside_hs_c',
 'metargleu_e',
 '1331tacr_c',
 'tcynt_m',
 'sphgn_r',
 'c10dc_e',
 'HC00900_c',
 'dcholcoa_c',
 'proglnpro_e',
 'acnamp_c',
 'fucfuc12gal14acglcgalgluside_hs_g',
 'Lpipecol_x',
 'sl_L_e',
 'CE2510_e',
 'M02449_c',
 'c8crn_c',
 'prist_e',
 'tethex3coa_x',
 'dmnoncoa_x',
 'strdnccoa_c',
 'ppa_e',
 'fucgalacgalfuc12gal14acglcgalgluside_hs_g',
 'estrone_r',
 'crglz_c',
 '3hlvstacid_e',
 'M00962_c',
 'dlnlcg_r',
 'lyspheile_c',
 'adrncoa_c',
 'dtmp_m',
 'M00101_m',
 'octdececoa_m',
 'ncam_c',
 'glyc3p_x',
 'gly_c',
 'acmana_c',
 'C11695_e',
 'lnlccoa_m',
 'tmndnccoa_r',
 'dcamp_c',
 'oretn_n',
 'smv_r',
 'asncyscys_c',
 'CE4988_m',
 'CE4801_m',
 'dolglcp_L_r',
 'tcynt_c',
 'xoltriol_m',
 'CE5786_e',
 'dmhptcoa_m',
 '3hpcoa_m',
 'acglcgal14acglcgalgluside_hs_g',
 'fpram_c',
 'lvst_c',
 'acgal_g',
 'CE2434_m',
 'M00806_m',
 '3m4hpga_c',
 'cholate_r',
 'thm_c',
 'mercplac_c',
 '4nph_c',

In [12]:

nx.write_gpickle(grafo_nx, "./results/graphs/NX_recon_graph.gpickle")


In [7]:
#import pandas as pd

feature_data = feature_data.sort_values(by=['label'])
feature_data.reset_index(drop=True, inplace=True)
flux_samples = flux_samples.sort_values(by=['label'])
flux_samples.reset_index(drop=True, inplace=True)
assert all(feature_data.label == flux_samples.label)

Labels = flux_samples.label



flux_samples.drop("label", axis=1, inplace=True)
feature_data.drop("label", axis=1, inplace=True)

In [8]:


blank_features = pd.DataFrame(
                    np.full((len(feature_data), list(grafo_nx.nodes).__len__()), 1e-5),  columns=list(grafo_nx.nodes)
                    )
blank_features.reset_index(drop=True, inplace=True)
assert len(blank_features) == len(feature_data) == len(flux_samples)
assert set(feature_data.columns).issubset(set(list(grafo_nx.nodes)))


In [9]:

blank_features_dict = blank_features.to_dict(orient="list")  
flux_samples_dict   = flux_samples.to_dict(orient="list")  
concentrations_dict = feature_data.to_dict(orient="list")  



In [10]:
import copy
#import pandas as pd
def update_df_features(base, new):
    
    common_vars  = list(set(base.keys()).intersection(set(new)))
    updated      = copy.deepcopy(base)
    updated.update({key: new[key] for key in common_vars})
    
    return updated

    
features_only_fluxes_dict         = update_df_features(blank_features_dict, flux_samples_dict)
features_only_concentrations_dict = update_df_features(blank_features_dict, concentrations_dict)
full_features_dict = copy.deepcopy(concentrations_dict)
full_features_dict.update(flux_samples_dict)  # = pd.concat([feature_data, flux_samples], axis= 1 )
features_completed_dict = update_df_features(blank_features_dict, full_features_dict)

In [11]:
assert flux_samples.r0399[Labels == 0].mean()    > 20
assert flux_samples.r0399[Labels == 1].mean()    < 1
assert feature_data.phe_L_c[Labels == 0].mean() < 50
assert feature_data.phe_L_c[Labels == 1].mean() > 500
assert all(features_completed_dict['r0399'] == features_only_fluxes_dict['r0399'] == flux_samples.r0399) 
assert all(features_completed_dict['phe_L_c'] == features_only_concentrations_dict['phe_L_c'] == feature_data.phe_L_c) 

In [12]:

def new_nx_from_dict(nx_G_in, feature_dict):
    
    nx_G        = copy.deepcopy(nx_G_in)    
    x_attribute = feature_dict #nx.get_node_attributes(nx_G, "x")
     
    nx.set_node_attributes(nx_G, x_attribute, 'x')
    
    len(nx_G.nodes(data=True)['r0399']['x']) == len(nx_G.nodes(data=True)['phe_L_c']['x'])
    assert nx_G.nodes(data=True)['phe_L_c']['x']  == feature_dict['phe_L_c']#.tolist() 
    assert nx_G.nodes(data=True)['r0399']['x']  == feature_dict['r0399']#.tolist() 

    
    
    return nx_G

nx_features_only_concentrations = new_nx_from_dict(grafo_nx, features_only_concentrations_dict)  
nx_features_only_fluxes         = new_nx_from_dict(grafo_nx, features_only_fluxes_dict)  
nx_features_features_completed  = new_nx_from_dict(grafo_nx, features_completed_dict)  


assert nx_features_only_concentrations.nodes(data=True)['r0399']['x']    != nx_features_only_fluxes.nodes(data=True)['r0399']['x'] 
assert nx_features_only_concentrations.nodes(data=True)['phe_L_c']['x']  != nx_features_only_fluxes.nodes(data=True)['phe_L_c']['x'] 
assert nx_features_features_completed.nodes(data=True)['phe_L_c']['x']  == nx_features_only_concentrations.nodes(data=True)['phe_L_c']['x'] 
assert nx_features_features_completed.nodes(data=True)['r0399']['x']    == nx_features_only_fluxes.nodes(data=True)['r0399']['x'] 

In [13]:
#grafo_nx_onlyConcen  = add_metabolite_concentration_features(grafo_nx, feature_data, feature_names)
#print(f"{set(feature_names.Recon3_ID) - set(list(grafo_nx_onlyConcen.nodes)).intersection(set(feature_names.Recon3_ID))} is not in Recon2")

pyg_graph_onlyConcen         = make_PYG_graph_from_grafo_nx(nx_features_only_concentrations)
pyg_graph_onlyFluxes         = make_PYG_graph_from_grafo_nx(nx_features_only_fluxes)
pyg_graph_Concen_plus_Fluxes = make_PYG_graph_from_grafo_nx(nx_features_features_completed)


In [15]:

pyg_graph_onlyConcen.y               = torch.tensor(Labels).reshape(len(Labels),1)
pyg_graph_onlyFluxes.y               = torch.tensor(Labels).reshape(len(Labels),1)
pyg_graph_Concen_plus_Fluxes.y       = torch.tensor(Labels).reshape(len(Labels),1)

torch.save(pyg_graph_onlyConcen, "./results/graphs/PYG_graph_only_Concen.pt")
torch.save(pyg_graph_onlyFluxes, "./results/graphs/PYG_graph_only_Fluxes.pt")
torch.save(pyg_graph_Concen_plus_Fluxes, "./results/graphs/PYG_graph_Concen_plus_Fluxes.pt")